### Unsupervised Learning

- ML, BN 들을 어떻게 나누고, 구분지어 생각할 수 있는지 [ML-overview](https://github.com/NamSahng/Summary/blob/master/Intro2AI%26ML/OverView-ML.ipynb)


- Instance에 대해 true value를 모르고, 제공해 주지 않음

- Discovering Clusters, Latent Factors, Graph Structures
    - ex)
        - Finding the representative topic words from text data
        - Finding the latent image from facial data
        - Completing the incomplete matrix of product-review score
        - Filtering the noise from the trajectory(궤도) data

    - Clustering: Estimating sets and affiliations of instances to the sets.
        - Latent Factor(내부 동력)에 대한 Optimal Assignment. 
    - Filtering: estimating underlying and fundamental signals from the mixture of signals and noises
        - CRF, MRF들이 filtering으로 사용가능.
        - 클러스터링에 집중해서 보자.



## A. K-means algorithm

- Setup K # of centroids(or prototypes) and cluster data points by the distance from the points to the nearest Centroid.
    - 문제를 보면 문제가 2가지(셋업하는 거 빼고)로 구성됨을 알 수 있다.
        - Centroid를 찾는 것.
        - Data를 centroid에 Assign하는 것.
    
   
- $ J = \sum_{n=1}^{N}\sum_{k=1}^{K} r_{nk} \lVert x_{n} - \mu_{k} \rVert^{2}   $ 를 Minimize by Optimizing
    - $ r_{nk} $: the Assignment of data points to clusters (centroid에 대한 assignment 정보 => 1 or 0)
    - $ \mu_{k} $: the Location of Centroids
        -  2 variables are interacting하기 때문에, Iterative Optimization.
   
   
### - Expectation and Maximization: 
- (맛보기로 보자, 뒤에 자세히)

- Expectation: $ r_{nk} $를 optimize
    - Expectation of the log-likelihood given the parameters
        - 이 방법이 Assgining the data points to teh nearest Centroid. 
    -  $ r_{nk} = \left\{ 0 , 1 \right\} $:
        - Discrete Variable이며, Logical choice(nearest centroid $ \mu_{k} $ for data point of $ x_{n} $

- Maximization: $ \mu_{k} $를 
    - Maximization of the parameters with respect to the likelihood
         - Update the centroid positions given assignment (Centroid 포지션을 다시 계산 )
     - $ \mu_{k} $: (위으 given $ r_{nk} $에 대해)
        - $ {{dJ}\over{d\mu_{k}}} =  {{d}\over{d\mu_{k}}}    \sum_{n=1}^{N}\sum_{k=1}^{K} r_{nk} \lVert x_{n} - \mu_{k} \rVert^{2} = {{d}\over{d\mu_{k}}} \sum_{n=1}^{N} r_{nk} \lVert x_{n} - \mu_{k} \rVert^{2}   $ : 미분했을 때, k에 맞지 않는 것들은 상수항으로 없어지므로
        - $ =  \sum_{n=1}^{N} -2 r_{nk} ( x_{n} - \mu_{k} )  \to $
        - $ \mu_{k} = {{  \sum_{n=1}^{N} -2 r_{nk}x_{n} }\over{  \sum_{n=1}^{N} -2 r_{nk} }} $ 즉 assign된 data들의 평균값이 된다.

### - Problems of K-means

- a) # of Cluster 가 확실하지않다. (베이지안 num parametric(?)를 사용하면 해결 가능)
- b) centroid의 초기값 (이것도 여러 스킬들이 있다. 여러 번하는 것, 평균잡아 등등)
- c) Euclidean Distance만을 고려한다. 
    - 모든 dimension에 대해 중요도가 같으므로 위험할 수 있다.
- d) Hard Clustering

- c와 d는 GMM으로 해결 가능하다.



## B. Gaussian Mixture Model


- 준비물1: Multinomial Distribution에서의 MLE estimation using Lagrange Method
    - latent factor를 다항분포의 random variable로 생각하면, N개의 선택지(latents, cluster)중 k 번째의 확률 $ \mu $를 구하기.
        - $ P(X|\mu) = \prod_{n=1}^{N} \prod_{k=1}^{K} \mu_{k} {}^{x_{nk}} = \prod_{k=1}^{K} \mu_{k} {}^{ \sum_{n=1}^{N} x_{nk}} = \prod_{k=1}^{K} \mu_{k} {}^{m_{k}}   $
        - 이것의 확률을 구하기 위해 MLE를 하면 문제는
        - Maximize $ P(X|\mu) =   \prod_{k=1}^{K} \mu_{k} {}^{m_{k}} $, 와 제약조건 $ \mu_{k} \geq 0 , \sum_{k} \mu_{k} = 1 $ 을 만족해야 한다.
        - 제약조건을 포함한 MLE를 풀기위해, Lagrange Multiplier를 도입해 풀면, 자세한 과정 (IE661-Week 8-Part 2-icmoon-ver-1.PPTX의 5pg)
        - binomial distribution에서 MLE의 값이 $ {{H}\over{H+T}} $ 였 듯이, $ \mu_{k} = {{m_{k}}\over{N}} $로 즉 (특정 선택지가 선택된 개수)/(선택지들의 수)가 된다.
        


- 준비물2: Multivariate Gaussian Distribution 에 대한 MLE estimation
    - 가우시안분포는 $ N(x|\mu,\sigma^{2}) = {{1}\over{\sqrt{2\pi\sigma^{2}}}} exp(- {{1}\over{2\sigma^{2}}} (x-\mu)^{2} ) $ 이며, 이를 다항 가우시안분포로 바꾸면 (single dimension to multi dimension)
    -  $ N(x|\mu,\Sigma) =  {{1}\over{{(2\pi)^{D/2}}}} {{1}\over{{|\Sigma|^{1/2}}}} exp( -{{1}\over{2}} (x-\mu)^{T} \Sigma^{-1} (x-\mu) )  $ 가 되고, 이를 log likelihood로 만들어 Trace trick(대각합)성질을 이용하면,
        - $ lnN(x|\mu,\Sigma) \propto -{{N}\over{2}}ln|\Sigma| -{{1}\over{2}} Tr \left[ \Sigma^{-1}  \sum_{n=1}^{N}((x-\mu)(x_{n}-\mu)^{T}) \right] $
    - 그리고 평균벡터와 공분산행렬에 대해 MLE Estimation을 하면,
        - $ {{d}\over{d\mu}} lnN(x|\mu,\Sigma) = 0 \to \hat{\mu} = {{\sum_{n=1}^{N} x_{n}}\over{N}} $
        - $ {{d}\over{d\Sigma}} lnN(x|\mu,\Sigma) = 0 \to \hat{\Sigma} = {{1}\over{N}} \sum_{n=1}^{N} (x-\hat{\mu})(x_{n}-\hat{\mu})^{T} $

    - cf)  Multivariate Gaussian Distribution의 covariance matrix의 성질 2x2에서
<img  src="./image/W8-1.PNG" width="70%">
        - 각 dimension에 대한 분산과 dimension간의 joint에 대한 분산
        - covariance와 correlation은 깊은 연관이 있다.



### - Mixture Model
- 위 두 준비물을 묶자
<img  src="./image/W8-2.PNG" width="80%">

- 맨 왼쪽의 데이터에 대한 히스토그램에 대해 그냥 normal로 fitting하면 잘 안되니까, source가 다른 3개에서 mixture되서 왔구나 생각
    - subpopulation 


- $ P(x) = \sum_{k=1}^{K} \pi_{k} N(x|\mu_{k},\sigma_{k})  $
    - 데이터에 대해 normal로 fitting을 하는데 mixing coefficients $ \pi_{k} $를 추가해여
    - Mixing Coefficients $  \pi_{k} $: 일종의 weighting coeff. Multinomial distribution으로 modelling, $ P(z_{k}) $로 모델링
    - Mixture Component $ N(x|\mu_{k},\sigma_{k}) $: Multivariate Gaussian Distribution으로 Modelling, $ P(x|z) $
    
- $ \to P(x) =  \sum_{k=1}^{K} P(z_{k}) P(x|z)    $



### - GMM

-  Datat point가 Multiple(여러 개의) Multivariate Gaussian Distribution의 mixture distribution이라고 가정하면
- $ P(x) = \sum_{k=1}^{K} P(z_{k}) P(x|z) =  \sum_{k=1}^{K} \pi_{k} N(x|\mu_{k},\sigma_{k}) $
    - Mixing Coefficients(selection variables): $ z_{k} $의 선택(selection)은 0,1로 hard하게 되어있지만, selection의 과정이 probabilistic(stochastic)하게 이루어진다.
        - $ z_{k} \in \left\{ 0,1 \right\} $, &nbsp; $ \sum_{k} z_{k} = 1 $, &nbsp; $ P(z_{k} = 1) = \pi_{k} $ &nbsp; $ \sum_{k}^{K} \pi_{k} = 1 $, &nbsp; $ 0 \leq \pi_{k} \leq  1 $
        - $ P(Z) = \prod_{k=1}^{K} \pi_{k} {}^{z_{k}} $
    - Mixture Component: k번째 latent일 때의 확률
        - $ P(X|z_{k}=1) =  N(x|\mu_{k},\boldsymbol{\Sigma_{k}}) $ 여기서 selection된 특정 mixture coef.것 만을 활용했다 해서 아래처럼 표현
        - $ \to P(X|Z) =  \prod_{k=1}^{K} N(x|\mu_{k},\boldsymbol{\Sigma_{k}})^{z_{k}} $ 
        
        
- Bayesian Network Notation
    - 파란색(점으로 하기도함): 파라미터임을 나타내며, 베이지안 버전으로 하려면 파란색들이 random variable의 되면서 위에 prior distribution이 위에 붙을 것이다.
    - z가 random variable로 표현, x는 검정원인 것이 관측된 값이기 때문에, latent가 N개이므로 plate notation
<img  src="./image/W8-3.PNG" width="20%">


- 관측된 given 데이터에서 zk는 어떻게 latent assign하는지. (위의 반대로)
    - $ \gamma(z_{nk}) \equiv p(z_{k}=1|x_{n}) = {{P(z_{k} = 1)P( x|z_{k} = 1)  }\over{\sum_{j=1}^{K} P(z_{j} = 1)P( x|z_{j} = 1)}} $
    - $ = {{ \pi_{k} N(x|\mu_{k},\boldsymbol{\Sigma_{k}})}\over{\sum_{j=1}^{K} \pi_{j} N(x|\mu_{j},\boldsymbol{\Sigma_{j}})}} $


- 그리고(따라서) 전체 데이터셋에 대한 이때, log-likelihood는 
    - $ lnP(X|\pi, \mu, \boldsymbol{\Sigma}) = \sum_{n=1}^{N} ln \left\{ \sum_{k=1}^{K} \pi_{k} N(x|\mu_{k},\boldsymbol{\Sigma_{k}})  \right\} $
        - K-means는 확률적이지 않으 거리로만 계산해서 optimize한 반면, 이번에는 확률의 framework에 넣었다.



#### - Expectation of GMM

- k-means의 Assign a data point to nearset cluster의 부분을
    - 이제는 확률로 계산해 구할 수가 있으며 그냥 확률 그대로 활용해 soft하게 그대로 활용할 수 있다.
    
- 위의 $ \gamma(z_{nk}) \equiv p(z_{k}=1|x_{n}) $ 를 이용해 assignment probability를 계산한다.
    - $ \gamma(z_{nk}) = {{P(z_{k} = 1)P( x|z_{k} = 1)  }\over{\sum_{j=1}^{K} P(z_{j} = 1)P( x|z_{j} = 1)}} = {{ \pi_{k} N(x|\mu_{k},\boldsymbol{\Sigma_{k}})}\over{\sum_{j=1}^{K} \pi_{j} N(x|\mu_{j},\boldsymbol{\Sigma_{j}})}}  $
    - given $ x, \pi, \mu, \boldsymbol{\Sigma} $ 에 대해  $ \gamma(z_{nk}) $ 계산

#### - Maximization of GMM

- 다시 given $ \gamma(z_{nk})  $ 를 활용해, $ \pi, \mu, \boldsymbol{\Sigma} $를 update
    - $ {{d}\over{d \mu}} lnP(X|\pi, \mu, \boldsymbol{\Sigma}) = 0 $
        - $ \to \hat{\mu_{k}} = {{\sum_{n=1}^{N}\gamma(z_{nk})x_{n}}\over{\sum_{n=1}^{N} \gamma(z_{nk}) }} $
    - $ {{d}\over{d \boldsymbol{\Sigma}}} lnP(X|\pi, \mu, \boldsymbol{\Sigma}) = 0 $
        - $ \to \hat{\boldsymbol{\Sigma_{k}}} =  {{\sum_{n=1}^{N}\gamma(z_{nk}) (x-\hat{\mu_{k}})(x_{n}-\hat{\mu_{k}})^{T}}\over{\sum_{n=1}^{N} \gamma(z_{nk}) }} $
        - Multivariate Gaussian Distribution의 MLE estimation of Covariance와 유사
    - $ \pi $ 는 제약조건이 있으므로, Lagrange Multiplier를 도입해 $ {{d}\over{d \pi }} lnP(X|\pi, \mu, \boldsymbol{\Sigma}) + \lambda (\sum_{k=1}^{K} \pi_{k}-1) = 0$
        - $ \to \lambda = -N $
        - $ \to \hat{\pi_{k}} = {{\sum_{n=1}^{N}\gamma(z_{nk})}\over{N}} $

<img  src="./image/W8-4.PNG" width="60%">

- 색깔을 보면 cluster 경계에서 soft하게 clustering 됨을 알 수 있다.


#### - Pros and Cons of GMM

- Pros:
    - Latent Factor에 대한 이해 (Unsupervised Learning의 Essense)
        - Learn more latent distribution
        - and more information
    - soft clustering으로 인한 Information loss를 줄임

- Cons:
    - 계산량 : covariance matrix 모델링은 빡세다. k-means 보다 넘 느림
    - local minimum: EM은 어쩔수 없다.
    - Deciding K: bayesian GMM 으로 만들고 bayesian num parametric으로 해결가능
    
   

### Relation between K-Means and GMM

- covariance matrix를 identity matrix로 가정해보자. 즉 개별 dimension 에서만 분산이 있고, covariance(correlation)은 없다고 생각.
    - let $ \boldsymbol{\Sigma_{k}} = \epsilon I $
    - 이 때, $ \epsilon $ 이 작아질 수록 hard assignment가 되며 0일 때는 hard assignment가 된다. (수식으로 사고실험을 해보면)
    - 즉 GMM은 K-means의 superset이며, Soft Assginment와 Covariance Matrix Learning이 가장 큰 차이임을 알 수 있다.
    
    
    
### EM ALGORITHM

- 즉, Latent의 유무가 Classfication: 과 Clustering의 차이이다.
    - Classfication : $ ln(P(X|\theta)) $
    - Clustering : $ \sum_{Z}P(X,Z|\theta) \to ln(\sum_{Z}P(X,Z|\theta))  $
        - 로그안에 summation이 있어서 optimize하기 불편하다.

- 그래서 이제 Z도 모르고 세타도 모르니 iterative하게 optimize하겠다.
    - 이것 끼리 interaction이 존재하기 때문에


- 위의 어려운 계산을 위한 Jensen's Inequality
    - Convex일 때, 함수를 계산하고 평균을 취한것이 평균을 취해 인풋으로 쓰는 것보다 더 크다.
<img  src="./image/W8-5.PNG" width="30%">


- Jensen's Inequality를 $ ln(\sum_{Z}P(X,Z|\theta)) $에 활용, log는 concave하니까
    - $ ln(\sum_{Z}P(X,Z|\theta)) = ln \left\{ \sum_{Z} q(Z) {{P(X,Z|\theta))}\over{q(Z)}} \right\} \geq \sum_{Z} q(Z) ln( {{P(X,Z|\theta))}\over{q(Z)}}) $
    -  $   =  \sum_{Z} q(Z) ln(P(X,Z|\theta)) - q(Z)ln(q(Z)) $ 
        - $ q(Z)ln(q(Z)) $가 엔트로피와 굉장히 닮았다. (q를 pdf 함수여야함)
        - $  \sum_{Z} q(Z) ln(P(X,Z|\theta)) $는 q(z)를 coef로 갖는 expectation 형태이다.
        - 즉 expectaion & entropy형태로 바꾸면,
    - $ = E_{q(Z)}ln(P(X,Z|\theta)) + H(q) $
        - 이를 $ Q(\theta,q) $라하고 이것이 lower bound of $ ln(\sum_{Z}P(X,Z|\theta)) $이 것이니까 이를 maximize 하게 optimize하는 것이 방법이겠다.
   - $ l(\theta) = ln(P(X|\theta)) = ln \left\{ \sum_{Z} q(Z) {{P(X,Z|\theta))}\over{q(Z)}} \right\}  $
   - $ Q(\theta,q) =  E_{q(Z)}ln(P(X,Z|\theta)) + H(q) $
        
- Maximizing Lower Bound.
    - $ln(\sum_{Z}P(X,Z|\theta))$의 lower bound , $ \sum_{Z} q(Z) ln( {{P(X,Z|\theta))}\over{q(Z)}}) $에 대해 $ P(X,Z|\theta)) $에 조건부확률의 정리를 이용해 식을 갖고 놀면, Inequility를 만드는 원인을 찾을 수 있다.
    - $ln(P(X|\theta)) \geq ln(P(X|\theta)) - \sum_{Z}\left\{ q(Z)ln{{q(Z)}\over{p(Z|X,\theta)}} \right\} $
    - 그 원인은 KL divergence of $ q(Z) $ and $ p(Z|X,\theta) $ 이고 이를 사라지게 하는 것이 tight하게 만들어 maximize할 수 있겠구나.
        - $ KL( q(Z) || p(Z|X,\theta) ) $ 이걸 0으로 만들면 optimize되겠다.        
    - Kullback-Leiber Divergence: 
       - 분포의 유사도, 차이를 측정하는 것, 같으면 0, 항상 0보다 큼
       - [PRML-정보이론](  https://github.com/NamSahng/Summary/blob/master/Pattern_Recognition%26Machine_Learning/1.6%20Information%20Theory.ipynb)
    - $ L(\theta,q) = ln(P(X|\theta)) - \sum_{Z}\left\{ q(Z)ln{{q(Z)}\over{p(Z|X,\theta)}} \right\} $   
   
- $ L(\theta,q) $의 KLD부분을 통해 q(z)를 어떻게 set해야 하는지 알려준다.
    - $ L(\theta,q) $의 $ ln(P(X|\theta)) $은 ** 특정시간 t에서 ** $ \theta $ 가 고정되면 $ ln(P(X|\theta)) $도 고정.
    - 그리고 KLD부분은 그 때의 $ \theta $를 받아 $ q^{t}(Z) = P(Z|X,\theta^{t}) $하면 0이된다.
    - 다시 여기서 구한 $q^{t}$를 이용해 $ Q(\theta,q) =  E_{q(Z)}ln(P(X,Z|\theta)) + H(q) $를 업데이트친다.
        - $ \theta^{t+1} = argmax_{\theta}Q(\theta,q^{t}) = argmax_{\theta} E_{q^{t}(Z)}ln(P(X,Z|\theta)) $
    - 이걸 반복!

- graphical하게 보면 다음과 같다
    - Local optimum에 빠질 수 있지만 practical리 잘 작동한다.
<img  src="./image/W8-6.PNG" width="80%">

- 정리

<img  src="./image/W8-7.PNG" width="80%">

- 여기에 시간이 추가된 HMM을 다음시간에 보자.


<br><br>
### Reference:

- 문일철 교수님, 인공지능 및 기계학습 개론 II, https://www.edwith.org/machinelearning2__17